In [9]:
#!/usr/bin/env python3.9 -m nuitka
# -*- coding: utf-8 -*-
import re
import os
import sys
import json
import time
import urllib3
import platform
import requests
import unicodedata
import configparser
import browser_cookie3
from pprint import pprint
from datetime import datetime
from playsound import playsound
from sdk.api.message import Message
from sdk.exceptions import CoolsmsException

urllib3.disable_warnings()

class Headers:
    headers_map = {
        "Accept": "application/json, text/plain, */*",
        "Content-Type": "application/json;charset=utf-8",
        "Origin": "https://vaccine-map.kakao.com",
        "Accept-Language": "en-us",
        "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 14_7 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 KAKAOTALK 9.4.2",
        "Referer": "https://vaccine-map.kakao.com/",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "Keep-Alive",
        "Keep-Alive": "timeout=5, max=1000"
    }
    headers_vacc = {
        "Accept": "application/json, text/plain, */*",
        "Content-Type": "application/json;charset=utf-8",
        "Origin": "https://vaccine.kakao.com",
        "Accept-Language": "en-us",
        "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 14_7 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 KAKAOTALK 9.4.2",
        "Referer": "https://vaccine.kakao.com/",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "Keep-Alive",
        "Keep-Alive": "timeout=5, max=1000"
    }

def load_cookie_from_chrome():
    global jar

    cookie_file = load_cookie_config()
    if cookie_file is False:
        return

    if cookie_file is None:
        cookie_path = None
        os_type = platform.system()
        if os_type == "Linux":
            # browser_cookie3 also checks beta version of google chrome's cookie file.
            cookie_path = os.path.expanduser(
                "~/.config/google-chrome/Default/Cookies")
            if os.path.exists(cookie_path) is False:
                cookie_path = os.path.expanduser(
                    "~/.config/google-chrome-beta/Default/Cookies")
        elif os_type == "Darwin":
            cookie_path = os.path.expanduser(
                "~/Library/Application Support/Google/Chrome/Default/Cookies")
        elif os_type == "Windows":
            cookie_path = os.path.expandvars(
                "%LOCALAPPDATA%/Google/Chrome/User Data/Default/Cookies")
        else:  # Jython?
            print("지원하지 않는 환경입니다.")
            close()

        if os.path.exists(cookie_path) is False:
            print("기본 쿠키 파일 경로에 파일이 존재하지 않습니다. 아래 링크를 참조하여 쿠키 파일 경로를 지정해주세요.\n" +
                  "https://github.com/SJang1/korea-covid-19-remaining-vaccine-macro/discussions/403")
            close()

    jar = browser_cookie3.chrome(
        cookie_file=cookie_file, domain_name=".kakao.com")

def check_user_info_loaded(try_count_for_check_user=0):
    global jar
#     print(jar)
    user_info_api = 'https://vaccine.kakao.com/api/v1/user'
    user_info_response = requests.get(
        user_info_api, headers=Headers.headers_vacc, cookies=jar, verify=False)
    user_info_json = json.loads(user_info_response.text)
    if user_info_json.get('error'):
        print("API에러.. 쿠키 다시 체크중({})".format(try_count_for_check_user), end="\r")
        # cookie.ini에 있는 쿠키가 유통기한 지났을 수 있다
        # 비교 위해서 cookie.ini 쿠키를 'prev_jar'에 저장한다
        prev_jar = jar 
        load_cookie_from_chrome()

        # 크롬 브라우저에서 새로운 쿠키를 찾았으면 다시 체크 시작 한다
        if prev_jar != jar:
            #  print('new cookie value from chrome detected')
            try_count_for_check_user += 1
            check_user_info_loaded(try_count_for_check_user)
            return

        print("사용자 정보를 불러오는데 실패하였습니다.")
        print("Chrome 브라우저에서 카카오에 제대로 로그인되어있는지 확인해주세요.")
        print("로그인이 되어 있는데도 안된다면, 카카오톡에 들어가서 잔여백신 알림 신청을 한번 해보세요. 정보제공 동의가 나온다면 동의 후 다시 시도해주세요.")
        close()
    else:
        user_info = user_info_json.get("user")
        for key in user_info:
            value = user_info[key]
            # print(key, value)
            if key != 'status':
                continue
            if key == 'status' and value == "NORMAL":
                print("사용자 정보를 불러오는데 성공했습니다.")
                break
            elif key == 'status' and value == "UNKNOWN":
                print("상태를 알 수 없는 사용자입니다. 1339 또는 보건소에 문의해주세요.")
                close()
            else:
                print("이미 접종이 완료되었거나 예약이 완료된 사용자입니다.")
                close(success=None)

def close(success=False):
    if success is True:
        playsound("tada.mp3")
        print("잔여백신 예약 성공!! \n 카카오톡지갑을 확인하세요.")
        send_lms("잔여백신 예약 성공!! 카카오톡지갑을 확인하세요.")
    elif success is False:
        print("오류와 함께 잔여백신 예약 프로그램이 종료되었습니다.")
    else:
        pass
    input("Press Enter to close...")
    sys.exit()

def send_lms(message):
    global hp_number, api_secret
    # set api key, api secret
    api_key = "NCS57256296AFC9B"

    ## 4 params(to, from, type, text) are mandatory. must be filled
    params = dict()
    params["type"] = "sms" # Message type ( sms, lms, mms, ata )
    params["to"] = hp_number # Recipients Number "01000000000,01000000001"
    params["from"] = hp_number # Sender number
    params["text"] = message # Message

    cool = Message(api_key, api_secret)
    try:
        response = cool.send(params)
        print("Success Count : %s" % response["success_count"])
        print("Error Count : %s" % response["error_count"])
        print("Group ID : %s" % response["group_id"])
        if "error_list" in response:
            print("Error List : %s" % response["error_list"])
    except CoolsmsException as e:
        print("Error Code : %s" % e.code)
        print("Error Message : %s" % e.msg)

def try_reservation(organization_code, vaccine_type):
    reservation_url = 'https://vaccine.kakao.com/api/v2/reservation'
    data = {"from": "Map", "vaccineCode": vaccine_type,
            "orgCode": organization_code, "distance": None}
    response = requests.post(reservation_url, data=json.dumps(
        data), headers=Headers.headers_vacc, cookies=jar, verify=False)

    f = open("response.txt", "w") # 파일 열기
    print(response.text, file=f) # 파일 저장하기
    f.close()

    response_json = json.loads(response.text)
    for key in response_json:
        value = response_json[key]
        if key != 'code':
            continue
        if key == 'code' and value == "NO_VACANCY":
            print("잔여백신 접종 신청이 선착순 마감되었습니다.")
            time.sleep(0.08)
        elif key == 'code' and value == "TIMEOUT":
            print("TIMEOUT, 예약을 재시도합니다.")
            retry_reservation(organization_code, vaccine_type)
        elif key == 'code' and value == "SUCCESS":
            print("백신접종신청 성공!!!")
            organization_code_success = response_json.get("organization")
            print(
                f"병원이름: {organization_code_success.get('orgName')}\t" +
                f"전화번호: {organization_code_success.get('phoneNumber')}\t" +
                f"주소: {organization_code_success.get('address')}")
            close(success=True)
        else:
            print("ERROR. 아래 메시지를 보고, 예약이 신청된 병원 또는 1339에 예약이 되었는지 확인해보세요.")
            print(response.text)
            close()

def retry_reservation(organization_code_success, vaccine_type):
    reservation_url = 'https://vaccine.kakao.com/api/v1/reservation/retry'

    data = {"from": "Map", "vaccineCode": vaccine_type,
            "orgCode": organization_code, "distance": None}
    response = requests.post(reservation_url, data=json.dumps(
        data), headers=Headers.headers_vacc, cookies=jar, verify=False)
    response_json = json.loads(response.text)
    for key in response_json:
        value = response_json[key]
        if key != 'code':
            continue
        if key == 'code' and value == "NO_VACANCY":
            print("잔여백신 접종 신청이 선착순 마감되었습니다.")
            time.sleep(0.08)
        elif key == 'code' and value == "SUCCESS":
            print("백신접종신청 성공!!!")
            organization_code_success = response_json.get("organization")
            print(
                f"병원이름: {organization_code_success.get('orgName')}\t" +
                f"전화번호: {organization_code_success.get('phoneNumber')}\t" +
                f"주소: {organization_code_success.get('address')}")
            close(success=True)
        else:
            print("ERROR. 아래 메시지를 보고, 예약이 신청된 병원 또는 1339에 예약이 되었는지 확인해보세요.")
            print(response.text)
            close()

def find_vaccine():
    global jar, search_time, reserve_run, except_list, vaccine_want, min_end_time, end_type
    global top_x, top_y, bottom_x, bottom_y, only_left
    try_count = 0
    not_want_hosp = []
    done = False
    time.sleep(search_time)

    url = 'https://vaccine-map.kakao.com/api/v3/vaccine/left_count_by_coords'
    data = {"bottomRight": {"x": bottom_x, "y": bottom_y}, "onlyLeft": "true", "order": "latitude",
            "topLeft": {"x": top_x, "y": top_y}}
    while True:
        if done is True:
            break
        else:
            try_count += 1
            try:
                response = requests.post(url, data=json.dumps(
                    data), headers=Headers.headers_map, cookies=jar, verify=False, timeout=5)

                try:
                    json_data = json.loads(response.text)
#                     pprint(json_data)
                    print("{}번째 시도중...".format(try_count), end="\r")

                    for x in json_data["organizations"]:
                        if done is True:
                            break
                        if x.get('orgName') in except_list:
                            pass
                        elif x.get('status') == "AVAILABLE" or x.get('leftCounts') != 0:
                            organization_code = x.get('orgCode')
                            check_organization_url = f'https://vaccine.kakao.com/api/v3/org/org_code/{organization_code}'
                            check_organization_response = requests.get(check_organization_url, headers=Headers.headers_vacc, cookies=jar,
                                                                       verify=False)
                            hosp = json.loads(check_organization_response.text)
                            end_time = int(hosp["organization"]["openHour"]["openHour"]["end"].split(":")[0])

                            # 실제 백신 남은수량 확인
                            vaccine_found_code = None
                            for vac_info in hosp.get("lefts"):
                                if vac_info.get('leftCount') != 0:
                                    if vac_info.get('vaccineName') in vaccine_want:
                                        if end_time >= min_end_time:
                                            if reserve_run:
                                                print("{}번째 시도중... {} : {} 백신을 {}개 발견했습니다.".format(try_count, hosp["organization"]["orgName"], vac_info.get('vaccineName'), vac_info.get('leftCount')))
                                                print("주소는 : {} 입니다.".format(hosp["organization"]["address"]))
                                                vaccine_found_code = vac_info.get('vaccineCode')
                                                done = True
                                                playsound("wow.wav")
                                                try_reservation(organization_code, vaccine_found_code)
                                                break
                                            else:
                                                print("{}번째 시도중... {} : {} 백신을 {}개 발견하였으나 예약은 진행하지 않습니다.".format(try_count, hosp["organization"]["orgName"], vac_info.get('vaccineName'), vac_info.get('leftCount')))
                                        else:
                                            print("{}번째 시도중... {} : {} 백신을 {}개 발견하였으나 시간이 맞지 않아 예약은 진행하지 않습니다.".format(try_count, hosp["organization"]["orgName"], vac_info.get('vaccineName'), vac_info.get('leftCount')))
                                    else:
                                        not_want_hosp.append(hosp["organization"]["orgName"])
                                        print("{}번째 시도중... 원하는 백신이 아니어서 패스합니다 : {}".format(try_count, hosp["organization"]["orgName"]), end=end_type)

                except json.decoder.JSONDecodeError as decodeerror:
                    print("JSONDecodeError : ", decodeerror)
                    print("JSON string : ", response.text)
                    close()
            except requests.exceptions.Timeout as timeouterror:
                print("Timeout Error : ", timeouterror)
            except requests.exceptions.SSLError as sslerror:
                print("SSL Error : ", sslerror)
                close()
            except requests.exceptions.ConnectionError as connectionerror:
                print("Connection Error : ", connectionerror)
                # See psf/requests#5430 to know why this is necessary.
                if not re.search('Read timed out', str(connectionerror), re.IGNORECASE):
                    close()
            except requests.exceptions.HTTPError as httperror:
                print("Http Error : ", httperror)
                close()
            except requests.exceptions.RequestException as error:
                print("AnyException : ", error)
                close()

            f = open("not_want_hosp.txt", "w") # 파일 열기
            print("\""+"\", \"".join(not_want_hosp)+"\"", file=f) # 파일 저장하기
            f.close()

def main_function():
    load_cookie_from_chrome()
#     check_user_info_loaded()
    find_vaccine()
    close()

# ===================================== run ===================================== #
jar = None
search_time = 0.2  # 잔여백신을 해당 시간마다 한번씩 검색합니다. 단위: 초
vaccine_want = ["화이자", "모더나"]
min_end_time = 0 #18일 경우 18시 포함 더 늦은 종료시간인 병원에 해당할 경우에만 예약함
hp_number = "" #문자발송을 위한 정보
api_secret = "" #문자발송을 위한 정보

#포항지역 좌표정보
top_x = "129.4369078955544"
top_y = "35.931968680267204"
bottom_x = "129.28111700916696"
bottom_y = "36.059338527137314"

#대구 율하동
# top_x = "128.67968519190245"
# top_y = "35.87748405678825"
# bottom_x = "128.71620567043774"
# bottom_y = "35.8579721781758"

only_left = "true"

# except_list = ["벧엘의원", "오천조내과의원", "오천중앙의원", "박응원미모아의원", "스마트가정의학과 의원", "기재만가정의학과의원", "청림연합의원", "삼성의원", "배순호의원", "신승호정형외과의원", "아산가정의학과의원", "김양식내과의원", "곽의원", "다솜의원", "신영호내과의원", "백철운소아청소년과의원", "송가정의학과의원"]
# end_type = "\r"

except_list = []
end_type = "\n"

reserve_run = True
if __name__ == '__main__':
    main_function()

<CookieJar[<Cookie __T_=1 for .accounts.kakao.com/>, <Cookie sticky-cookie=67b3fa1cef8528b0e69bece45c571903 for .cs.kakao.com/>, <Cookie aid=7de1ae75d74d496a81273eb1312d281f for .ds.kakao.com/>, <Cookie aid_ts=1611041769804 for .ds.kakao.com/>, <Cookie TIARA=beLxH5EL2A26V7Nr3BoJdrKF7M855s.xPnVbB1BzPm-x51s.OFBVOrhJCAFG5yOhA-6RUbrG1RfXuCQWUCzBE.h9MJe5QpB_87c1CwD4qp50 for .kakao.com/>, <Cookie _T_ANO=JlZiFt+TtuprxGCxxsQxMqtSNMO9kZ4rJkgLAzhohitz7S6qewLQyDPEhffId/xQNwOYCpcC9kWoEzRE9M/Cw/pQj+GTRiKHin8tAV/ey8Ijr7IxCBRdb0u/g6jnoCBk3X5cCRFM5tfhhHdyTox8z5tpzTi90genXux+9xQ/XVP24Q7hYcXWEUarUNUigiQj2+pKl/qAblDW1ymeZcI0UOXpc/CrvUK0h2azF/r40FgAEdiXhW1DMS7zaU7F1BYYju2dsixDfEZMSD6d3efWc/QCuVA334q0DLqcaswBGnT2pPgmWSKM8unHvXAH3wsqsyR4rANM0Dn71u+uGNv/UA== for .kakao.com/>, <Cookie _kadu=ExyN2HbV2nIEuUYJ_1628473777998 for .kakao.com/>, <Cookie _karb=nWVP4WERIvYpsps5_1628482885363 for .kakao.com/>, <Cookie _karmt=uarqxrojtxSZnKyWdIkaLCxTh5-dIiV4zjaG9l8SQm8f9iJlDnCX0rTAXJD02mwM for .kakao.com/>, <Cookie _kar

KeyboardInterrupt: 